# Neural Networks With TensorFlow
## Stable implementation

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow import variance_scaling_initializer # He-initializer
from jupyter_tf_graph import show_graph

In [19]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.reshape(-1, 28 * 28) / 255.0
X_test = X_test.reshape(-1, 28 * 28) / 255.0

## Training a Fully Connected Deep Neural Network

In [40]:
tf.reset_default_graph()

outfile = "./tf_logs/vanilla_dnn"
n_inputs = 28 * 28
n_hidden = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=None, name="y")

he_init = variance_scaling_initializer()
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden, activation=tf.nn.elu,
                              kernel_initializer=he_init, name="hidden1")
    
    hidden2 = tf.layers.dense(hidden1, n_hidden, activation=tf.nn.elu
                              ,name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden, activation=tf.nn.elu,
                             name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden, activation=tf.nn.elu,
                             name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden, activation=tf.nn.elu,
                             name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs,
                             name="logits")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y,
        logits=logits
    )
    loss = tf.reduce_mean(xentropy, name="loss")

    
alpha = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(alpha)
    train_step = optimizer.minimize(loss)

with tf.name_scope("metrics"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    writer = tf.summary.FileWriter(outfile)
    accuracy_writer = tf.summary.scalar("accuracy", accuracy)

saver = tf.train.Saver()
init = tf.global_variables_initializer()

In [43]:
epochs = 200
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        sess.run(train_step, feed_dict={X: X_train, y: y_train})
        
        train_acc, tboard_acc = sess.run([accuracy, accuracy_writer],
                                         feed_dict={X: X_train, y:y_train})
        
        writer.add_summary(tboard_acc, epoch)
        end = "\n" if epoch % 20 == 0 else "\r"
        print(f"@Epoch {epoch}, accuracy = {train_acc:0.2%}", end=end)

@Epoch 0, accuracy = 33.70%
@Epoch 20, accuracy = 85.79%
@Epoch 40, accuracy = 92.57%
@Epoch 60, accuracy = 94.71%
@Epoch 80, accuracy = 95.87%
@Epoch 100, accuracy = 96.76%
@Epoch 120, accuracy = 97.48%
@Epoch 140, accuracy = 98.05%
@Epoch 160, accuracy = 98.51%
@Epoch 180, accuracy = 95.44%
